In [1]:
##############################
#Training loop for NNs       #
#Maintainer: Christopher Chan#
#Version: 0.2.4              #
#Date: 2022-03-13            #
##############################

import os
import sys
import torch
import pathlib
import time
import re
import PIL
import random
import ray
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch.nn as nn
import segmentation_models_pytorch as smp
from ray import tune
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.schedulers import HyperBandForBOHB
from scipy import ndimage
from PIL import Image
from torch import optim
from tqdm import tqdm
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torch.utils.tensorboard import SummaryWriter
from Networks import Five_UNet, Four_UNet
from dataloader import BuildingDataset
%matplotlib inline

device = (torch.device("cuda") if torch.cuda.is_available()
          else torch.device("cpu"))

print(f"Training on device {device}.")

#td_KBYSamp = os.path.abspath("/home/chris/Dropbox/HOTOSM/SAMPLE/td_KBY")
#td_DZKSamp = os.path.abspath("/home/chris/Dropbox/HOTOSM/SAMPLE/td_DZK")
#td_DZKNSamp = os.path.abspath("/home/chris/Dropbox/HOTOSM/SAMPLE/td_DZKN")

td_KBY10 = os.path.abspath("/home/mnt/HOTOSM_data/Kakuma10cm/Kalobeyei/td_KBY")
td_DZK10 = os.path.abspath("/home/mnt/HOTOSM_data/Dzaleka10cm/td_DZK")
td_DZKN10 = os.path.abspath("/home/mnt/HOTOSM_data/Dzaleka_N10cm/td_DZKN")


Training on device cuda.


## Change class label from 2 to 1 for binary

for root, dirs, filename in os.walk(os.path.join(td_DZK10, "LBL")):
    for i in filename:
        if i.endswith(".png"):
            png = Image.open(root + "/" + i)
            px = png.load()

            for x in range(png.size[0]):
                for y in range(png.size[1]):
                    if px[x, y] == 2:
                        px[x, y] = 1
                        
                png.save(root + "/" + i, "PNG")

### Train Val Test split

In [2]:

# Below is a set of relatively complex functions which:
# Perform the train, val, test split at a rounded ratio of 62%, 27%, and 10% based on each sets of imagery
# This will be followed by first pseudo changing the name of _LBL_ to _IMG_ to match the split imagery
# Lastly, once the correct LBL files are matched, 

def tvt_split(td):
    
    img_ls = []
    
    for root, dirs, filename in os.walk(os.path.join(td, "IMG")):
        for i in filename:
            if i.endswith(".png"):
                img_ls.append(root + "/" + i)
        
        img_ls = BuildingDataset(img_ls, _)
        
        train_IMG, val_IMG, test_IMG = random_split(img_ls.png_dir, [int(round(0.6 * len(img_ls.png_dir))),
                                                                     int(round(0.3 * len(img_ls.png_dir))),
                                                                     int(round(0.1 * len(img_ls.png_dir)))])
        
        return train_IMG, val_IMG, test_IMG

DZK_train, DZK_val, DZK_test = tvt_split(td_DZK10)
KBY_train, KBY_val, KBY_test = tvt_split(td_KBY10)
DZKN_train, DZKN_val, DZKN_test = tvt_split(td_DZK10)

##################
# TOO COMPLICATED#
##################

#def match_LBL(td, imgs):
#    
#    lbl_ls = []
#    img_ls = []
#    match_ls = []
#    
#    imgs = list(imgs)
#    
#    for root, dirs, filename in os.walk(os.path.join(td, "LBL")):
#        for j in filename:
#            if j.endswith(".png"):
#                ps_name = j.rsplit("_LBL_")[0] + "_IMG_" + j.rsplit("_LBL_")[1] # Parse the string, PSEUDO-CHANGE _LBL_ to _IMG_
#                lbl_ls.append(ps_name)
#    
#    for k in imgs:
#        names = os.path.basename(k)
#        img_ls.append(names)
#        
#    def common(a, b):
#        a_set = set(a)
#        b_set = set(b)
#        if (a_set & b_set):
#            return (a_set & b_set)
#        else:
#            print("No common elements")
#            
#            
#    match_ls = common(img_ls, lbl_ls)
#        
#    match_ls = [(root + "/" + n.replace("_IMG_", "_LBL_")) for n in match_ls] # Change the _IMG_ back to _LBL_
#    
#    print("For the selected dataset of {0}, There are: {1} images, {2} labels, and {3} matching image/label pairs.".format(os.path.basename(td), len(img_ls), len(lbl_ls), len(match_ls)))
#    
#    return match_ls

#########################################
# Assign matched LBL to new LBL datasets#
#########################################

#DZKLBL_Train = match_LBL(td_DZK, DZK_train)
#DZKLBL_Val = match_LBL(td_DZK, DZK_val)
#DZKLBL_Test = match_LBL(td_DZK, DZK_test)
#DZKNLBL_Train = match_LBL(td_DZKN, DZKN_train)
#DZKNLBL_Val = match_LBL(td_DZKN, DZKN_val)
#DZKNLBL_Test = match_LBL(td_DZKN, DZKN_test)
#KBYLBL_Train = match_LBL(td_KBY, KBY_train)
#KBYLBL_Val = match_LBL(td_KBY, KBY_val)
#KBYLBL_Test = match_LBL(td_KBY, KBY_test)

############
# Try again#
############

TrainLBL_ls = []
ValLBL_ls = []
TestLBL_ls = []

TrainIMG_ls = list(DZK_train + KBY_train + DZKN_train)
ValIMG_ls = list(DZK_val + KBY_val + DZKN_val)
TestIMG_ls = list(DZK_test + KBY_test + DZKN_test)

for i in TrainIMG_ls:
    i = re.sub("IMG", "LBL", i, count = 2)
    TrainLBL_ls.append(i)

for i in ValIMG_ls:
    i = re.sub("IMG", "LBL", i, count = 2)
    ValLBL_ls.append(i)

for i in TestIMG_ls:
    i = re.sub("IMG", "LBL", i, count = 2)
    TestLBL_ls.append(i)

Train = BuildingDataset(png_dir = TrainIMG_ls,
                        lbl_dir = TrainLBL_ls)

Val = BuildingDataset(png_dir = ValIMG_ls,
                      lbl_dir = ValLBL_ls)

Test = BuildingDataset(png_dir = TestIMG_ls,
                       lbl_dir = TestLBL_ls)

assert len(Train.png_dir) == len(Train.lbl_dir)

print("Total images and labels pair in DataLoader: {0}".format(len(Train.png_dir) + len(Val.png_dir) + len(Test.png_dir)))

print("Concatenated TRAINING images and labels pair: {0} :".format(len(Train.png_dir)))
#for x, y in zip(Train.png_dir, Train.lbl_dir):    
#    print(f"Image: {x}", f"Label: {y}")

print("Concatenated VALIDATION images and labels pair: {0} :".format(len(Val.png_dir)))
#for x, y in zip(Val.png_dir, Val.lbl_dir):
#    print(f"Image: {x}", f"Label: {y}")

print("Concatenated TESTING images: {0} and labels pair: {0} :".format(len(Test.png_dir)))
#for x, y in zip(Test.png_dir, Test.lbl_dir):
#    print(f"Image: {x}", f"Label: {y}")


Total images and labels pair in DataLoader: 3947
Concatenated TRAINING images and labels pair: 2368 :
Concatenated VALIDATION images and labels pair: 1184 :
Concatenated TESTING images: 395 and labels pair: 395 :


In [3]:
# Trimmed down version
def rayHPO_loop(n_epochs, optimizer, model, xp_name,
                loss_fn, in_channels, out_channels, train_loader,
                checkpoint_freq):

    model = model.train()

    for epoch in tqdm(range(1, n_epochs + 1)):
         
        log_dir = os.path.abspath("/home/mnt/HOTOSM_data/log")
        checkpointdir = os.path.abspath("/home/mnt/HOTOSM_data/ray_checkpoints")
        loss_train = 0.0

        for i, (imgs, labels) in tqdm(enumerate(train_loader), total = len(train_loader)):
            imgs = imgs.to(device = device, dtype = torch.float32)
            labels = labels.to(device = device, dtype = torch.float32)

            prediction = model(imgs)
            loss = loss_fn(prediction.squeeze(0), labels.squeeze(0))
            
            optimizer.zero_grad()
            loss_train += loss.item()
            loss.backward()
            optimizer.step()

            global_step = epoch * len(train_loader) + i

        # Validation
        #if epoch % val_freq == 0:
        #    
        #    model = model.eval()
        #    val_loss = 0.0

        #    with torch.no_grad():
        #        for i, (imgs, labels) in tqdm(enumerate(val_loader), total = len(val_loader)):
        #            imgs = imgs.to(device = device, dtype = torch.float32)
        #            labels = labels.to(device = device, dtype = torch.float32)

        #            prediction = model(imgs)
        #            val_loss += loss_fn(prediction.squeeze(0), labels.squeeze(0))
        #            assert val_loss.requires_grad == False

        if epoch % checkpoint_freq == 0:
            with tune.checkpoint_dir(step = global_step) as checkpoint_dir:
                checkpoint_dir = checkpointdir
                os.makedirs(os.path.join(checkpoint_dir, xp_name), exist_ok = True)
                checkpoint_file = os.path.join(checkpoint_dir, xp_name, xp_name + "_iter_" + str(global_step).zfill(6) + ".pth")
                model_state = {}

                model_state = model.state_dict()

                state = {"Model:": model_state, "Epoch:": epoch, "Steps:": global_step}
                torch.save(state, checkpoint_file)

            tune.report(loss = (loss_train / global_step))
    
    print("Finished Training")

###########################################
###########################################
###########################################

def training_loop(n_epochs, optimizer, model, xp_name,
                  loss_fn, in_channels, out_channels, train_loader,
                  val_loader, val_freq, checkpoint_freq):

    model = model.train()

    for epoch in tqdm(range(1, n_epochs + 1)):
        
        log_dir = os.path.abspath("/home/mnt/HOTOSM_data/log")
        checkpointdir = os.path.abspath("/home/mnt/HOTOSM_data/checkpoints")
        writer = SummaryWriter(os.path.join(log_dir, xp_name))

        loss_train = 0.0

        for i, (imgs, labels) in tqdm(enumerate(train_loader), total = len(train_loader)):
            imgs = imgs.to(device = device, dtype = torch.float32)
            labels = labels.to(device = device, dtype = torch.float32)

            prediction = model(imgs)
            loss = loss_fn(prediction.squeeze(0), labels.squeeze(0))

            
            optimizer.zero_grad()
            loss_train += loss.item()
            loss.backward()
            optimizer.step()

            global_step = epoch * len(train_loader) + i

            if global_step % 10 == 0:
                writer.add_scalar("Train/Loss", loss.item(), global_step = global_step)
                writer.add_scalar("Batch/Loss", loss_train/len(train_loader), global_step = global_step)
        # Validation

        if epoch % val_freq == 0:
            
            model = model.eval()
            val_loss = 0.0

            with torch.no_grad():
                for i, (imgs, labels) in tqdm(enumerate(val_loader), total = len(val_loader)):
                    imgs = imgs.to(device = device, dtype = torch.float32)
                    labels = labels.to(device = device, dtype = torch.float32)

                    prediction = model(imgs)
                    val_loss += loss_fn(prediction.squeeze(0), labels.squeeze(0))
                    assert val_loss.requires_grad == False

#                    if i == 0:
#                        writer.add_images("Val/Sample_LBL", labels[:, :, :, :].squeeze(0), dataformats = "CHW",
#                                          global_step = global_step)
#
#                        if out_channels > 1:
#                            writer.add_images("Val/Sample_conf_1", prediction[:, 0, :, :].squeeze(0), 
#                                              global_step = global_step, dataformats = "CHW")
#                            writer.add_images("Val/Sample_conf_2", prediction[:, 1, :, :].squeeze(0), 
#                                              global_step = global_step, dataformats = "CHW")
#
#                            confidence = prediction[:, 0, :, :] - prediction[:, 1, :, :]
#                            writer.add_images("Val/Sample_conf", confidence.squeeze(0), 
#                                              global_step = global_step, dataformats = "CHW")
#
#                            prediction = torch.argmax(prediction, 1).cpu().detach().numpy()
#                        else:
#                            writer.add_images("Val/Sample_conf", confidence.squeeze(0), 
#                                              global_step = global_step, dataformats = "CHW")
#
#                            prediction = (torch.sigmoid(prediction) > 0.5)
#
#                        writer.add_images("Val/Sample_pred", prediction.squeeze(0), 
#                                          global_step = global_step, dataformats = "CHW")
#
                writer.add_scalar("Val/Loss", val_loss.item(), global_step = global_step)


        if epoch % checkpoint_freq == 0:
            os.makedirs(os.path.join(checkpointdir, xp_name), exist_ok = True)
            checkpoint_file = os.path.join(checkpointdir, xp_name, xp_name + "_iter_" + str(global_step).zfill(6) + ".pth")
            model_state = {}

            model_state = model.state_dict()

            state = {"Model:": model_state, "Epoch:": epoch, "Steps:": global_step}
            torch.save(state, checkpoint_file)

In [4]:
Net = smp.Unet(encoder_name = 'efficientnet-b1', encoder_depth = 5, encoder_weights=None, decoder_use_batchnorm=True, decoder_channels=(256, 128, 64, 32, 16), decoder_attention_type=None, in_channels=3, classes=1, activation="sigmoid", aux_params=None)
qubvel_weights = os.path.abspath("/home/chris/Dropbox/HOTOSM/qubvel_UNet/weights/stage3/effb1-f0/checkpoints/best.pth")
qubvel_weights = torch.load(qubvel_weights, map_location = device)
Net.load_state_dict(qubvel_weights["state_dict"])

#Net = smp.Unet(encoder_name='efficientnet-b1', encoder_depth = 4, encoder_weights = "imagenet", decoder_use_batchnorm = True, decoder_channels=(128, 64, 32, 16), decoder_attention_type = None, in_channels= 3 , classes = 1, activation = "sigmoid", aux_params = None)

#Net = Four_UNet()

#Net = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', in_channels=3, out_channels=1, init_features=32, pretrained=True)

Net = Net.to(device = device)

n_params = [p.numel() for p in Net.parameters() if p.requires_grad == True]

print(Net)
print('Trainable parameters in current model:', n_params)

Unet(
  (encoder): EfficientNetEncoder(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_

### RayTune Hyperparameter Optimisation experiments

In [7]:
# Ray tune Hyperparameter Optimsisation experiment using RayHPO_loop
xp_name = "2368:1184oc1_EB1-UNet-IMN_RAY_HYPERBANDBOHB_ep100_BCE"

config = {"lr": np.exp(np.random.uniform(1e-5, 1e-2)),
          "wd": np.exp(np.random.uniform(1e-6, 1e-2)),
          "batch_size": int(np.random.choice([4, 6]))}

#config = {"lr": tune.loguniform(1e-5, 1e-2),
#          "wd": tune.loguniform(1e-6, 1e-2),
#          "batch_size": tune.choice([4, 6])}

def main(num_samples = 30, max_epochs = 15, gpus_per_trial = 1):

    search_algo = TuneBOHB(mode = "min",
                           metric = "mean_loss")

    scheduler = HyperBandForBOHB(time_attr = "training_iteration",
                                 metric = "mean_loss",
                                 mode = "min",
                                 max_t = max_epochs)
    
    Loop = rayHPO_loop(n_epochs = max_epochs, 
                       optimizer = torch.optim.Adam(Net.parameters(), lr = config["lr"], weight_decay = config["wd"]),
                       model = Net,
                       in_channels = 3,
                       out_channels = 1,
                       xp_name = xp_name,
                       loss_fn = nn.BCELoss(reduction = 'mean'),
                       train_loader = DataLoader(Train, batch_size = config["batch_size"], shuffle = True),
                       checkpoint_freq = 3)

    result = tune.run(tune.with_parameters(Loop,
                                           resources_per_trial = {"cpu": 4, "gpu": gpus_per_trial},
                                           config = config,
                                           num_samples = num_samples,
                                           search_alg = search_algo,
                                           scheduler = scheduler))

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
    
    return best_trial

main()
torch.cuda.empty_cache()

100%|██████████| 592/592 [03:56<00:00,  2.50it/s]
Session not detected. You should not be calling `checkpoint_dir` outside `tune.run` or while using the class API. 
  File "/home/chris/.conda/envs/HOTOSM_gpu/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/chris/.conda/envs/HOTOSM_gpu/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/chris/.conda/envs/HOTOSM_gpu/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/chris/.conda/envs/HOTOSM_gpu/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/home/chris/.conda/envs/HOTOSM_gpu/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
    self.io_loop.start()
  File "/home/chris/.conda/envs/HOTOSM_gpu/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyn

Finished Training


ValueError: `tune.with_parameters() only works with function trainables or classes that inherit from `tune.Trainable()`. Got type: <class 'NoneType'>.

In [ ]:
torch.cuda.empty_cache()

print(best_trial)

NameError: name 'best_trial' is not defined

In [ ]:
xp_name = "2368:1184oc1_qubvel-UNet_lr5e-3_wd1e-5_b6_ep500_BCE"

training_loop(n_epochs = 500,
              optimizer = torch.optim.Adam(Net.parameters(), lr = 5e-3, weight_decay = 1e-5),
              model = Net,
              in_channels = 3,
              out_channels = 1,
              xp_name = xp_name,
              loss_fn = nn.BCELoss(reduction = 'mean'),
              train_loader = DataLoader(Train, batch_size = 6, shuffle = True),
              val_loader = DataLoader(Val, batch_size = 6 , shuffle = True),
              checkpoint_freq = 10,
              val_freq = 10)

torch.cuda.empty_cache()

In [6]:
torch.cuda.empty_cache()

## Prediction on Test dataset

In [ ]:
# plot a sample
test_pair = random.choice(Test)

fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
RGB = plt.imshow(test_pair[0].detach().cpu().numpy().transpose(1, 2, 0))
ax.set_title('RGB')

ax = fig.add_subplot(1, 2, 2)
LBL = plt.imshow(test_pair[1].detach().cpu().numpy().transpose(1, 2, 0))
ax.set_title('LBL')

In [ ]:
xp_name = "2368:1184oc1_EB1-UNet-IMN_lr1e-3_wd1e-5_b8_ep500_BCE"

Net = smp.Unet(encoder_name = 'efficientnet-b1', encoder_depth = 4, encoder_weights=None, decoder_use_batchnorm=True, decoder_channels=(128, 64, 32, 16), decoder_attention_type=None, in_channels=3, classes=1, activation="sigmoid", aux_params=None)
checkpoint_dir = os.path.abspath("/home/mnt/HOTOSM_data/checkpoints")
NN_weights = os.path.join(checkpoint_dir, xp_name, "2368:1184oc1_EB1-UNet-IMN_lr1e-3_wd1e-5_b8_ep500_BCE_iter_100935.pth")
NN_weights = torch.load(NN_weights, map_location = device)

Net.load_state_dict(NN_weights["Model:"])
Net.to(device = "cpu")
Net.eval()

img, lbl = test_pair[0], test_pair[1]
img = img.to(device = "cpu")
lbl = lbl.to(device = "cpu")
print(img.shape, lbl.shape)

with torch.no_grad():
    prediction = Net(img.unsqueeze(0))
    prediction = prediction.squeeze(0)

    print(prediction.shape)

    pred_BOOL = (prediction > 0.5).to(torch.float32)

    # Dice Loss
    dice_pred = pred_BOOL.sum(dim = [0, 1, 2])
    dice_lbl = lbl.sum(dim = [0, 1, 2])
    dice_score = 0
    dice_score += (2 * (pred_BOOL * lbl).sum(dim = [0, 1, 2]) + 1e-8) / (dice_pred + dice_lbl + 1e-8)
    dice_score = dice_score.detach().cpu().numpy()

    # True-Positive, True-Negative, False-Positive
    TP = (pred_BOOL * lbl).sum(dim = [0, 1, 2])
    FN = ((1 - pred_BOOL) * lbl).sum(dim = [0, 1, 2])
    FP = (pred_BOOL * (1 - lbl)).sum(dim = [0, 1, 2])
    IoU = (TP / (TP + FN + FP))

print("True Positive = {0}, False Negative = {1}, False Positive = {2}, IoU = {3}".format(TP, FN, FP, IoU))

# Plotting the RGB, Prediction, and Confidence
fig = plt.figure(figsize = (15, 6))
ax = fig.add_subplot(1, 4, 1)
fig.tight_layout(w_pad = 3)
fig.suptitle("EfficientNet-B1 U-Net ImageNet (transfer-untuned) | Dice Score: {0}".format(dice_score))
RGB = plt.imshow(test_pair[0].detach().cpu().numpy().transpose(1, 2, 0))
ax.set_title('RGB')

ax = fig.add_subplot(1, 4, 2)
PILlbl = lbl.detach().cpu().numpy().transpose(1, 2, 0)
plt.imshow(PILlbl)
ax.set_title('Label')

ax = fig.add_subplot(1, 4, 3)
PILpred = prediction.detach().cpu().numpy().transpose(1, 2, 0)
plt.imshow(PILpred, cmap = "brg")
plt.colorbar(boundaries = np.arange(0.5, 1.0, 0.1), ticks = np.arange(0.5, 1.0, 0.1),
             orientation = "horizontal")
ax.set_title('Prediction')

ax = fig.add_subplot(1, 4, 4)
PILbin = pred_BOOL.detach().cpu().numpy().transpose(1, 2, 0)
plt.imshow(PILbin, cmap = "hot")
ax.set_title('Binary Segmentation')


In [ ]:
# Loading testing batch
test_loader = DataLoader(Test, batch_size = 1, shuffle = True)
Net.to(device = device)

idx = []
Dice_ls = []
TP_ls = []
FN_ls = []
FP_ls = []
IoU_ls = []

for i, (img, lbl) in tqdm(enumerate(test_loader)):
    img = img.to(device = device)
    lbl = lbl.to(device = device)

    with torch.no_grad():
        prediction = Net(img)
        prediction = prediction.squeeze(0)
        lbl = lbl.squeeze(0)

        pred_BOOL = (prediction[:, 0:1] > 0.5).to(torch.float32)

        # Dice Loss
 # Dice Loss
        dice_pred = pred_BOOL.sum(dim = [0, 1, 2])
        dice_lbl = lbl.sum(dim = [0, 1, 2])
        dice_score = 0
        dice_score += (2 * (pred_BOOL * lbl).sum(dim = [0, 1, 2]) + 1e-8) / (dice_pred + dice_lbl + 1e-8)
        dice_score = dice_score.detach().cpu().numpy()
        Dice_ls.append(dice_score)

        # True-Positive, True-Negative, False-Positive
        
        TP = (pred_BOOL * lbl).sum(dim = [0, 1, 2])
        FN = ((1 - pred_BOOL) * lbl).sum(dim = [0, 1, 2])
        FP = (pred_BOOL * (1 - lbl)).sum(dim = [0, 1, 2])
        IoU = (TP / (TP + FN + FP))

        TP = TP.detach().cpu().numpy()
        FN = FN.detach().cpu().numpy()
        FP = FP.detach().cpu().numpy()
        IoU = IoU.detach().cpu().numpy()
        
        idx.append(i)
        TP_ls.append(TP)
        FN_ls.append(FN)
        FP_ls.append(FP)
        IoU_ls.append(IoU)

df = {"id": idx,
      "Dice_score": Dice_ls,
      "True_Positive": TP_ls,
      "False_Negative": FN_ls,
      "False_Positive": FP_ls,
      "IoU": IoU_ls}
      
results_csv = pd.DataFrame(df)
#print("mean True Positive = {0}, mean False Negative = {1}, mean False Positive = {2}, mean IoU = {3}".format(np.mean(TP_ls), np.mean(FN_ls), np.mean(FP_ls), np.mean(IoU_ls)))

In [ ]:
print(results_csv.info(), results_csv["Dice_score"].describe(), results_csv.head())